<a href="https://colab.research.google.com/github/tmskss/Flan-T5-K8S-QA/blob/main/flan_t5_k8s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is used to fine-tune flan-t5-base model on k8s question-command pairs gathered from k8s documentation

## Installing base requirements

In [1]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

  Using cached huggingface_hub-0.18.0-py3-none-any.whl (301 kB)


In [2]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

## Loading the dataset

In [3]:
dataset = load_dataset("Kristofy/k8s-kubectl")

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
def preprocess_function(examples):
    """tokenize the text, and set the labels"""

    inputs = [doc for doc in examples['question']]
    inputs = tokenizer(inputs, max_length=256, truncation=True, padding='max_length', return_tensors='pt')

    labels = tokenizer(text_target=examples['code'], max_length=256, truncation=True, padding='max_length', return_tensors='pt')

    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'labels': labels.input_ids,
    }


In [7]:
# Map the preprocessing function across our dataset
tokenized_dataset_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_dataset_eval = dataset['validate'].map(preprocess_function, batched=True)

Map:   0%|          | 0/3161 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

## Define metrics with ROUGE

In [9]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    print('[ROUGE] - Calculating metrics')

    # Apply only necessary processing without duplicating data
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute Rouge scores without unnecessary data copies
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    print('[ROUGE] - Metric calculation done')
    return result

## Set up training args and start training

In [8]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-base-k8s-question-code",
    evaluation_strategy="steps",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    num_train_epochs=5,
    push_to_hub=True
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    data_collator=None
)

# Train the model
trainer.train()

Step,Training Loss,Validation Loss
500,0.561000,0.035498
1000,0.031500,0.010901
1500,0.015700,0.006889
2000,0.007900,0.005116
2500,0.005700,0.003869
3000,0.003500,0.003480
3500,0.002400,0.003303


TrainOutput(global_step=3955, training_loss=0.07957126126729433, metrics={'train_runtime': 775.2311, 'train_samples_per_second': 20.387, 'train_steps_per_second': 5.102, 'total_flos': 5411294977720320.0, 'train_loss': 0.07957126126729433, 'epoch': 5.0})

In [9]:
trainer.push_to_hub("first finetune")

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

'https://huggingface.co/tmskss/flan-t5-base-k8s-question-code/tree/main/'

In [10]:
from transformers import AutoModelForSeq2SeqLM

eval_model = AutoModelForSeq2SeqLM.from_pretrained("tmskss/flan-t5-base-k8s-question-code")

In [17]:
# Define your text input
input_text = "List all  pods in Kubernetes"

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to('cuda')
eval_model.to('cuda')

# Generate text based on the input
output_ids = eval_model.generate(input_ids, max_length=100, num_return_sequences=1, num_beams=8)

# Decode the generated output
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)


kubectl get pods
